In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

%matplotlib inline

In [ ]:
no_ratings = pd.read_csv("df_NO_RATINGS.csv")
ratings = pd.read_csv("movies_df5.csv")
budget = pd.read_csv("budget_df.csv")

no_ratings = no_ratings.drop(['Unnamed: 0'], axis=1)
ratings = ratings.rename(index=str, columns = {'Unnamed: 0': 'imdb_id', '0': 'imdb_rating'})
budget = budget.rename(index=str, columns = {'Unnamed: 0': 'imdb_id', '0': 'budget'})


In [ ]:
movies_df = pd.merge(no_ratings, ratings, on="imdb_id")

movies_df = movies_df.dropna()

movies_df = pd.merge(movies_df, budget, on="imdb_id")

movies_df = movies_df.dropna()

In [ ]:
movies_df = pd.read_csv("movies_df_raw.csv")
movies_df = movies_df.drop(['Unnamed: 0'], axis=1)

In [ ]:
# Create test dataframe to gauge percentage of data lost if we drop NAs.

test = movies_df.dropna()

(len(movies_df) - len(test)) / len(movies_df)

# 3% data lost, ok to drop NAs

movies_df = movies_df.dropna()

In [ ]:
# Remove symbols in budget column
movies_df['budget'] = movies_df['budget'].str.replace(',', '')
movies_df['budget'] = movies_df['budget'].str.replace('$', '')
movies_df['budget'] = movies_df['budget'].str.replace("'", '')
movies_df['budget'] = movies_df['budget'].str.replace(']', '')
movies_df['budget'] = movies_df['budget'].str.replace('[', '')
movies_df['budget'] = movies_df['budget'].str.replace(']', '');

In [ ]:
# Remove foreign film by looking for foreign currency in budget
movies_df = movies_df[movies_df['budget'] != '€4000000']
movies_df = movies_df[movies_df['budget'] != 'रू2500000000']
movies_df = movies_df[movies_df['budget'] != 'रू700000000']
movies_df = movies_df[movies_df['budget'] != 'TRL2000000']
movies_df = movies_df[movies_df['budget'] != '￦10000000000']
movies_df = movies_df[movies_df['budget'] != '¥370000000']
movies_df = movies_df[movies_df['budget'] != '€12300000']

In [ ]:
# Convert appropriate columns to int and check type of each column
movies_df['runtime'] = pd.Series(movies_df['runtime'], dtype='int64')
movies_df['num_productionmanagers'] = pd.Series(movies_df['num_productionmanagers'], dtype='int64')
movies_df['num_stunts'] = pd.Series(movies_df['num_stunts'], dtype='int64')
movies_df['budget'] = pd.Series(movies_df['budget'], dtype='int64')
#movies_df['imdb_rating'] = pd.to_numeric(movies_df['imdb_rating'])

In [ ]:
# Convert release date to datetime and create release month column
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'])
movies_df['release_month'] = movies_df['release_date'].dt.month

In [ ]:
# Convert genre into list
movies_df['genre'] = movies_df['genre'].str.replace('"', '')
movies_df['genre'] = movies_df['genre'].str.replace('[', '')
movies_df['genre'] = movies_df['genre'].str.replace(']', '')
movies_df['genre'] = movies_df['genre'].str.split(',')

In [ ]:
# Remove first idex because it has over 6000 genre's (scraping error)
movies_df = movies_df.iloc[1:]

In [ ]:
# Finding number of genres to gauge how many dummy variables should be made by get_dummies

genre = movies_df['genre']

from itertools import chain

genre_list = list(chain.from_iterable(genres))

g = []

for genre in genre_list:
    genre = genre.replace("'", '')
    genre = genre.strip()
    g.append(genre)
    
print(set(g))

len(set(g))

In [ ]:
# Replace genres column with cleaner version
clean = []

for row in movies_df['genre']:
    g = []
    for genre in row:
        genre = genre.replace("'", '')
        genre = genre.strip()
        g.append(genre)
    clean.append(g)

movies_df['genres'] = clean

movies_df.drop(['genre'], axis=1)

In [ ]:
# Create dummy variables from genre

from sklearn.preprocessing import MultiLabelBinarizer

df_test = movies_df

s = df_test['genres']

mlb = MultiLabelBinarizer()

genres_df = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df_test.index)

genres_df.shape

# Shape of data frame with dummy variables match num of unique genres

In [ ]:
# Append dummy variables to data frame

df_test = pd.concat([movies_df, genres_df], axis=1)
df_test.to_csv("movies_df_clean_w_genre_dummy.csv")

### Cleaning recommendation data

In [ ]:
recs = pd.read_csv("recs.csv")

recs.head()
recs = recs.rename(index=str, columns={"Unnamed: 0": "imdb_id", "0": "genre", "1":"runtime","2":"release_date","3":"num_cast","4":"num_directors","5":"num_writers","6":"num_producers","7":"num_productionmanagers","8":"num_stunts","9":"num_productioncompanies", "10":"num_distributors","11":"budget"})

In [ ]:
recs['runtime'] = pd.Series(recs['runtime'], dtype='int64')
recs['num_productionmanagers'] = pd.Series(recs['num_productionmanagers'], dtype='int64')
recs['num_stunts'] = pd.Series(recs['num_stunts'], dtype='int64')
recs['budget'] = pd.Series(recs['budget'], dtype='int64')
recs['release_date'] = pd.to_datetime(recs['release_date'])
recs['release_month'] = recs['release_date'].dt.month

In [ ]:
recs['genre'] = recs['genre'].str.replace('"', '')
recs['genre'] = recs['genre'].str.replace('[', '')
recs['genre'] = recs['genre'].str.replace(']', '')
recs['genre'] = recs['genre'].str.split(',')

In [ ]:
clean = []
for row in recs['genre']:
    g = []
    for genre in row:
        genre = genre.replace("'", '')
        genre = genre.strip()
        g.append(genre)
    clean.append(g)
    
recs['genres'] = clean

recs.drop(['genre'], axis=1)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
df_test = recs
s = df_test['genres']
mlb = MultiLabelBinarizer()
genres_df = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df_test.index)
recs = pd.concat([recs, genres_df], axis=1)
recs = recs.drop(['imdb_id','genre','genres'], 1)

In [ ]:
selected_columns = list(recs2.columns.values)

In [ ]:
recs = recs.rename(index=str, columns={"Science Fiction": "Science_Fiction"})

In [ ]:
d = pd.DataFrame('0', index=range(11), columns=['Crime','Family','Fantasy','Horror','Music','Mystery','Romance','Thriller','War','Western'])

In [ ]:
test2 = pd.concat([recs,d],axis=1)

In [ ]:
recs1 = recs.apply(pd.to_numeric)

In [ ]:
recs2 = recs1.fillna(0)

recs2 = recs2.drop(['index'],1)

recs2.to_csv("recs.csv")